### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
import os
import functools
import operator
from collections import Counter, OrderedDict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

plt.ion()
%matplotlib inline
%config InlineBackend.figure_format='retina'

#### Local Imports

In [3]:
# Paths
from chestxray.config import (PANDA_PATH,
                              MODEL_PATH,
                              PANDA_IMGS,
                              PANDA_MASKS,
                              TRAIN_CSV)
# Competition related config
from chestxray.config import CFG

# Misc
from chestxray.misc import seed_torch

# # Datasets
# from chestxray.datasets import get_transforms, TilesTrainDataset, LazyTilesDataset, img_to_tiles, make_tiles, PatchTrainDataset

# # Viz
# from chestxray.visualize import (show_from_ids, show_batch, imshow, 
#                                  plot_classes_preds, reverse_show_img, 
#                                  plot_confusion_matrix)

# # Nets
from chestxray.nets import aggregate
# from chestxray.bit_net import ResNetV2, get_weights

# from chestxray.model_utils import trainable_params, cce_loss_at_init, check_final_linear_bias_init, init_last_layer_bias

# # Losses
# from chestxray.losses import LabelSmoothSoftmaxCEV1, TopkCrossEntropy

# from chestxray.layers import gem

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
print(torch.cuda.get_device_properties(device))

_CudaDeviceProperties(name='GeForce RTX 2070 SUPER', major=7, minor=5, total_memory=7979MB, multi_processor_count=40)


### Load Pretrained Model

In [6]:
enet_backbone = 'efficientnet-b0'

In [7]:
model = EfficientNet.from_pretrained(enet_backbone)

Loaded pretrained weights for efficientnet-b0


In [8]:
model.to(device)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [20]:
# del model._avg_pooling, model._dropout

In [10]:
model._global_params

GlobalParams(batch_norm_momentum=0.99, batch_norm_epsilon=0.001, dropout_rate=0.2, num_classes=1000, width_coefficient=1.0, depth_coefficient=1.0, depth_divisor=8, min_depth=None, drop_connect_rate=0.2, image_size=224)

In [11]:
model._dropout

Dropout(p=0.2, inplace=False)

In [9]:
x = torch.rand(2, 8, 3, 128, 128, device=device)

In [22]:
batch_size, num_patch, C, H, W = x.shape
x = x.view(-1, C, H, W)  # x -> bs*num_patch x C x H x W

In [23]:
x.shape

torch.Size([16, 3, 128, 128])

In [24]:
out = model.extract_features(x)

In [25]:
out.shape

torch.Size([16, 1280, 4, 4])

In [26]:
out = aggregate(out, batch_size, num_patch)

In [27]:
out.shape

torch.Size([2, 2560])

In [31]:
model._fc = nn.Sequential(
                OrderedDict(
                    [("cls_logit", nn.Linear(2 * 1280, CFG.target_size))]
                )
            )
model._fc.to(device)

Sequential(
  (cls_logit): Linear(in_features=2560, out_features=6, bias=True)
)

In [32]:
out = model._fc(out)

In [34]:
out.shape

torch.Size([2, 6])

In [37]:
model._fc.cls_logit

Linear(in_features=2560, out_features=6, bias=True)

### Build PatchEnet

In [13]:
class PatchEnetModel(nn.Module):
    def __init__(self, backbone='efficientnet-b0', n=CFG.target_size, pretrained=True):
        super().__init__()
        assert backbone in ['efficientnet-b0']
        
        if pretrained:
            self.model = EfficientNet.from_pretrained(backbone)
            
        num_ftrs = self.model._fc.in_features
        if CFG.model_cls == "deep":
            self.model._fc = nn.Sequential(
                OrderedDict(
                    [
                        ("cls_fc", nn.Linear(2 * num_ftrs, 512)),  # agregate use concat pooling, so *2
                        ("cls_bn", nn.BatchNorm1d(512)),
                        ("cls_relu", nn.ReLU(inplace=True)),
                        ("cls_logit", nn.Linear(512, CFG.target_size)),
                    ]
                )
            )
        elif CFG.model_cls == "one_layer":
            self.model._fc = nn.Sequential(
                OrderedDict(
                    [("cls_logit", nn.Linear(2 * num_ftrs, CFG.target_size))]
                )
            )
        del self.model._avg_pooling

    def forward(self, x):
        batch_size, num_patch, C, H, W = x.shape

        x = x.view(-1, C, H, W)  # x -> bs*num_patch x C x H x W
        x = self.model.extract_features(x)  # x -> bs*num_patch x C(Maps) x H(Maps) x W(Maps)

        x = aggregate(x, batch_size, num_patch)
        x = self.model._dropout(x)
        x = self.model._fc(x)
        return x

In [16]:
m = PatchEnetModel()
m.to(device)

Loaded pretrained weights for efficientnet-b0


PatchEnetModel(
  (model): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (

In [17]:
out = m(x)

In [18]:
out

tensor([[-1.8754, -0.4886,  1.2803,  0.9677, -1.2637, -1.3190],
        [-2.6603,  2.5725, -1.4255, -0.8157,  0.1718, -1.1945]],
       device='cuda:0', grad_fn=<AddmmBackward>)